# Conway's Game of Life

Authors: Edwin Weill & Brad Green  
Due Date: November 29th  

This iPython notebook serves as the project code for the final project in MATH 8650 (Data Structures).  

__Project Description__: Conway's game of life is a cellular automaton devised by John Horton Conway in 1970.
The "game" is structured so that the evolution is only based on the initial state, meaning no user input is needed.  Initial configurations can be created randomly or by creating known patterns with particular properties.  

A set of rules is derived that mimics growth of a colony of some biological organisms.  In most cases, the "game" is played on a two-dimensional grid which contains "dead" and "living" cells.  The following are a small subset of the rules that govern the evolution of the "game".  

- __Reproduction__: If a "dead" cell is surrounded by exactly 3 "living" cells, it become a "living" cell
- __Underpopulation__: If a "living" cell is surrounded by fewer than two "living" cells, it dies.
- __Overpopulation__: If a "living" cell is surrounded by more than three "living" cells, it dies.
- __Stasis__: If a "living" cell is surrounded by two or three "living" cells, it survives.

### Import necessary libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Set up display for animation

In [2]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

### Conway Game of Life Grid Class

In [3]:
class ConwayGOLGrid():
    """
    Represents a grid in the Conway's Game of Life problem where
    each of the cells contained in the grid may be either alive or
    dead for any given state.
    """
    def __init__(self, width=100, height=100):
        self.width, self.height = width, height
        self.cells = []

### Conway Game of Life Cell Class

In [4]:
class ConwayGOLCell():
    """
    Represents a cell in the Conway's Game of Life problem where
    a cell can either be alive or dead and the next state of the
    cell is based on the states of the immediate (8) neighbors.
    """
    def __init__(self, x, y, alive=False):
        """
        Create information for the given cell including the x and
        y coordinates of the cell, whether it is currently alive
        or dead, it's neighbors, and its current color.
        
        Parameters: x, y give the coordinate of the cell in grid
                    alive gives the current state of the cell
        Returns: None
        """
        self.x, self.y = x, y
        self.alive = alive
        self.neighbors = [(x-1,y-1), (x, y-1), (x+1, y-1),
                          (x-1,y  ),           (x+1, y  ),
                          (x-1,y+1), (x, y+1), (x+1, y+1)]
        self.color = (255,255,255)
        
    def spawn(self):
        """
        Changes the state of a cell from dead to alive.  Assumes
        that the cell is dead to be changed to alive (no need to
        modify if already alive).
        
        Parameters: None
        Returns: None
        """
        assert self.alive==False
        self.alive = True
        
    def die(self):
        """
        Changes the stat of a cell from alive to dead.  Assumes
        that the cell is alive to be changed to dead (no need to 
        modify if already dead).
        
        Parameters: None
        Returns: None
        """
        assert self.alive==True
        self.alive = False

### Animate Grid